In [1]:
import gzip
import math
import statistics as stats
import random
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn import linear_model
import json
import regex as re

In [2]:
#def parse(path):
#    g = gzip.open(path, 'r', encoding = 'utf-8')
#    for l in g:
#        yield json.loads(l)
#data = list(parseData("Video_Games.json.gz"))

In [3]:
#def parseData(fname):
#    for l in open(fname):
#        yield eval(l)
#        
#data = list(parseData("Video_Games.json"))

In [4]:
#import json
#with open('Video_Games.json') as f:
#   data = json.load(f)

In [5]:
df = pd.read_json('Video_Games.json.gz', lines=True)

In [6]:
df[:5]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN


## 1. EDA

In [57]:
data = df.sample(n = 400000)
data[:5]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
2523669,5,False,"01 12, 2015",APZYKY0QLRWOB,B00HEZCTBW,Cliff,A good game. The story is impressive with the ...,A good game. The story is impressive with the ...,1421020800,3,NaN,NaN
1792480,5,True,"10 28, 2015",A1BGWIYX5KSTEL,B00TEDK8FQ,Lone Wolf,I like it. It's very spacious and even though ...,Good purchase.,1445990400,NaN,NaN,NaN
2309637,3,True,"01 31, 2016",A2FFFWIOYLUFDM,B00175GC3Q,Brandon Danley,wish it had trophies,Three Stars,1454198400,NaN,"{'Edition:': ' Standard', 'Platform:': ' PlayS...",NaN
1759954,5,True,"08 29, 2015",A2S3LBYN4TC9TA,B00R7WKIH4,Jake Brighindi,Fit controllers amazingly. Adds color to plain...,Five Stars,1440806400,NaN,{'Color:': ' Camo-Grip-Series'},NaN
308183,4,True,"08 31, 2011",A9PQ4LL82MTP9,B000FRU1UM,jkimod,he plays online with a couple of his friends. ...,my 14 y.o. son seems to like it,1314748800,NaN,{'Format:': ' Video Game'},NaN


**------------This needs to be cited-------------** <br>
Jianmo Ni, Jiacheng Li, Julian McAuley
Empirical Methods in Natural Language Processing (EMNLP), 2019

The dataset we have chosen to work with are video game reviews on Amazon from 2014. There are 2,565,349 reviews in total, however for the purpose of computation, we have sampled 100,000 reviews. The data consists of 12 features: overall rating, verified (binary), time of review (raw), reviewer ID, product ID, name of reviewer, text of review, summary of product, time of review (unix), helpfulness votes (likes), product style, and attached image. 

Lets take a look at the data type for each column, and how many non-null values there are. 

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400000 entries, 2523669 to 175635
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         400000 non-null  int64 
 1   verified        400000 non-null  bool  
 2   reviewTime      400000 non-null  object
 3   reviewerID      400000 non-null  object
 4   asin            400000 non-null  object
 5   reviewerName    399956 non-null  object
 6   reviewText      399745 non-null  object
 7   summary         399866 non-null  object
 8   unixReviewTime  400000 non-null  int64 
 9   vote            69187 non-null   object
 10  style           205971 non-null  object
 11  image           4476 non-null    object
dtypes: bool(1), int64(2), object(9)
memory usage: 37.0+ MB


Lets add some features, like length of review text and length of summary, and replace NAN values with 0 for votes to perform some basic statistics.

In [59]:
data = data[['overall', 'reviewTime', 'reviewText', 'summary', 'unixReviewTime', 'vote']]

data['vote'] = data['vote'].replace(np.nan, 0).str.replace(',', '').astype(float)
data['reviewText'] = data['reviewText'].replace(np.nan, '')
data['summary'] = data['summary'].replace(np.nan, '')

data['len_reviewText'] = [len(x) for x in data['reviewText']]
data['len_summary'] = [len(x) for x in data['summary']]

In [60]:
data[['overall', 'vote', 'len_reviewText', 'len_summary']].describe()

,overall,vote,len_reviewText,len_summary
count,400000.000000,69187.000000,400000.000000,400000.000000
mean,4.021913,8.297874,401.701430,23.415335
std,1.405282,26.797440,825.585574,18.747273
min,1.000000,2.000000,0.000000,0.000000
25%,3.000000,2.000000,55.000000,10.000000
50%,5.000000,4.000000,152.000000,16.000000
75%,5.000000,7.000000,399.000000,31.000000
max,5.000000,3744.000000,32538.000000,281.000000


In [ ]:
#Histogram on months, extract months for each review and bar plot count per months**
#Bivariative: 

## 2. Predictive Task (Baseline)

In [61]:
wordCount = defaultdict(int)
for d in data['reviewText']:
    for w in re.sub(r'[^\w\s]',' ', d.lower()).split():
        #w = stemmer.stem(w)
        wordCount[w] += 1

In [62]:
most_common = sorted(wordCount.items(), key=lambda x:x[1], reverse = True)
most_common[:10]

[('the', 1590562),
 ('and', 825225),
 ('i', 780432),
 ('to', 770011),
 ('a', 742327),
 ('it', 709300),
 ('is', 518230),
 ('of', 511323),
 ('you', 472437),
 ('game', 465339)]

In [63]:
words = [w[0] for w in most_common[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [64]:
def feature(datum):
    feat = [0]*len(wordSet)
    if type(datum) == float:
        print('float here', datum)
    words = re.sub(r'[^\w\s]',' ', datum.lower()).split()
    for w in words:
        if not (w in wordSet): continue
        feat[wordId[w]] += 1
    #feat.append(1)
    return feat

In [65]:
X = [feature(d) for d in data['reviewText']]
y = [d for d in data['overall']]

In [66]:
clf = linear_model.Ridge(1.0, fit_intercept=True) # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_

In [67]:
test_data = df.sample(n = 100000)

test_data['vote'] = test_data['vote'].replace(np.nan, 0).str.replace(',', '').astype(float)
test_data['reviewText'] = test_data['reviewText'].replace(np.nan, '')
test_data['summary'] = test_data['summary'].replace(np.nan, '')

In [68]:
test_x = [feature(d) for d in test_data['reviewText']]
test_y = [d for d in test_data['overall']]
predictions = clf.predict(test_x)

In [69]:
mse = []
for overall, pred in zip(test_y, predictions):
    mse.append((overall - pred)**2)

sum(mse) / len(mse)

1.3220734588625265

In [70]:
#1.31971075
#1.3025469
#1.3220734588625265